# THE EFFECT OF PURE METAL'S ATOMIC PROPERTIES AND SURFACE CHARACTERISTICS ON ITS WORK FUNCTION: AN ANALYSIS USING SUPPORT VECTOR REGRESSION MODEL


## Requirement

This prediction  model is coded using Python 3.13.9. All the libraries that required can be installed using this cell below

In [ ]:
%pip install -r requirements.txt

In [ ]:
from mp_api.client import MPRester
from pymatgen.core.periodic_table import Element
from pymatgen.core import Structure, Lattice
from tqdm.auto import tqdm
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, root_mean_squared_error as rmse
import shap
import PyALE
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

MP_API_KEY = "oaqaUyUUgOtqC6jLsILaljDuuUEvrX89"

## Data Acquisition

### Data from Materials Project

In [ ]:
with MPRester(MP_API_KEY) as mpr:
    # Memilih material logam yang dibentuk oleh 1 unsur dan teramati lewat eksperimen
    summary_docs = mpr.materials.summary._search(
        is_metal = True,
        theoretical = False,
        nelements = [1, 1],
        fields = ["material_id", "nsites", "volume", "structure", "symmetry"]
    )
    structures = {str(doc.material_id): doc.structure for doc in summary_docs}

    # Mengubah data struktur ke sel konvensional
    conventional = {mid: s.to_conventional() for mid, s in structures.items()}

    material_id = [doc.material_id for doc in summary_docs]

    # Menyeleksi material dari summary_docs yang sifat permukaannya telah dihitung nilainya
    surface_properties_docs = mpr.materials.surface_properties.search(
        material_ids = material_id,
        fields = ["material_id", "pretty_formula", "surfaces"]
    )

    # Membuat list dari unsur-unsur yang telah dikumpulkan
    unique_elements = list({doc.pretty_formula for doc in surface_properties_docs})

### Data from pymatgen

In [ ]:
def valence_electrons_count(conf:str)->int: # Fungsi untuk menghitung banyaknya elektron valensi dari setiap unsur
    patterns = r'(\d+)([spdfgh])(\d{1,2})'
    subshells = re.findall(patterns,str(conf))
    if not subshells:
        return 0

    parsed_subshells = []
    for n, l, e in subshells:
        try:
            parsed_subshells.append((int(n), l, int(e)))
        except ValueError:
            continue

    if not parsed_subshells:
        return 0

    max_n=max(n for n, l, e in parsed_subshells)

    valence = 0
    for n, l, e in parsed_subshells:
        if n == max_n:
            valence+=e
        if l == 'd' and n == max_n - 1:
            valence+=e
        if l == 'f' and n == max_n - 2:
            valence+=e
        if l == 'g' and n == max_n - 3:
            valence+=e
        if l == 'h' and n == max_n - 4:
            valence+=e

    return valence

sifat_atomik = []

for simbol in tqdm(unique_elements):
    try:
        unsur = Element(simbol)

        config = unsur.electronic_structure

        data_unsur = {
            'formula_pretty': simbol,
            'atomic_number': unsur.Z,
            'atomic_radius': unsur.atomic_radius,
            '1st_ionization_energy_eV': unsur.ionization_energy, # Ionisasi pertama
            'electron_affinity_eV': unsur.electron_affinity,
            'electronegativity': unsur.X, # Skala Pauling
            'valence_electrons': valence_electrons_count(config),
            'Youngs_modulus': unsur.youngs_modulus,
            'shear_modulus': unsur.rigidity_modulus,
            'bulk_modulus': unsur.bulk_modulus
        }

        # 4. Tambahkan dictionary ke list utama
        sifat_atomik.append(data_unsur)

    except Exception as e:
        # Menangani jika ada data yang hilang di pymatgen (jarang terjadi)
        print(f"Gagal mengambil data Pymatgen untuk unsur '{simbol}': {e}")

### Create Dataframe using pandas + Feature Engineering

In [ ]:
print(summary_docs)

In [ ]:
print(conventional)

In [ ]:
summary_docs_list = []
for doc in summary_docs:
    summary_docs_list.append({
        "material_id": doc.material_id,
        "symmetry_crystal_system": doc.symmetry.crystal_system if doc.symmetry else None,
        "symmetry_symbol": doc.symmetry.symbol if doc.symmetry else None,
        "number_of_atoms": doc.nsites,
        "unit_cell_volume": doc.volume,
        "lattice_a": doc.structure.lattice.a if doc.structure and doc.structure.lattice else None,
        "lattice_b": doc.structure.lattice.b if doc.structure and doc.structure.lattice else None,
        "lattice_c": doc.structure.lattice.c if doc.structure and doc.structure.lattice else None,
        "lattice_alpha": doc.structure.lattice.alpha if doc.structure and doc.structure.lattice else None,
        "lattice_beta": doc.structure.lattice.beta if doc.structure and doc.structure.lattice else None,
        "lattice_gamma": doc.structure.lattice.gamma if doc.structure and doc.structure.lattice else None
    })

print(summary_docs_list)

In [ ]:
for doc_entry in summary_docs_list:
    material_id = str(doc_entry['material_id'])
    if material_id in conventional:
        struct = conventional[material_id]
        doc_entry.update({
            "unit_cell_volume": struct.volume,
            "number_of_atoms": len(struct),
            "lattice_a": struct.lattice.a,
            "lattice_b": struct.lattice.b,
            "lattice_c": struct.lattice.c,
            "lattice_alpha": struct.lattice.alpha,
            "lattice_beta": struct.lattice.beta,
            "lattice_gamma": struct.lattice.gamma
        })

print(summary_docs_list)

In [ ]:
summary_docs_df = pd.DataFrame(summary_docs_list)

summary_docs_df

In [ ]:
def classify_structure(row):
    # Membersihkan input
    system = str(row['symmetry_crystal_system']).split(':')[-1].replace("'>", "").strip()
    symbol = str(row['symmetry_symbol'])

    match (system, symbol): # Menggunakan match-case pada tuple (system, symbol)

        # Kubik
        case ('Cubic', s) if s.startswith('F'):
            return 'FCC'
        case ('Cubic', s) if s.startswith('I'):
            return 'BCC'
        case ('Cubic', s) if s.startswith('P'):
            return 'Simple Cubic'

        # Heksagonal
        case ('Hexagonal', 'P6_3/mmc'):
            return 'HCP' # Space group spesifik untuk HCP
        case ('Hexagonal', 'P6/mmm'):
            return 'Primitive Hexagonal'

        # Tetragonal
        case ('Tetragonal', s) if s.startswith('I'):
            return 'Body-Centered Tetragonal (BCT)'

        # Orthorhombic
        case ('Orthorhombic', s) if s.startswith('C'):
            return 'Base-Centered Orthorhombic'

        # Monoclinic
        case ('Monoclinic', s) if s.startswith('C'):
            return 'Base-Centered Monoclinic'

        # Trigonal
        case ('Trigonal', s) if s.startswith('R'):
            return 'Rhombohedral'

        # Default
        case _:
            return 'Other'

summary_docs_df['structure_type'] = summary_docs_df.apply(classify_structure, axis=1)
summary_docs_df["atomic_density_3d"] = summary_docs_df["number_of_atoms"] / summary_docs_df["unit_cell_volume"]
summary_docs_df["c_over_a_ratio"] = summary_docs_df["lattice_c"] / summary_docs_df["lattice_a"]
summary_docs_df["b_over_a_ratio"] = summary_docs_df["lattice_b"] / summary_docs_df["lattice_a"]

summary_docs_df

In [ ]:
cols = summary_docs_df.columns.tolist()

structure_type_col = cols.pop(cols.index('structure_type'))
atomic_density_3d_col = cols.pop(cols.index('atomic_density_3d'))
c_over_a_ratio_col = cols.pop(cols.index('c_over_a_ratio'))
b_over_a_ratio_col = cols.pop(cols.index('b_over_a_ratio'))

symmetry_symbol_idx = cols.index('symmetry_symbol')
lattice_a_idx = cols.index('lattice_a')

cols.insert(symmetry_symbol_idx , structure_type_col)
cols.insert(symmetry_symbol_idx + 2, atomic_density_3d_col)
cols.insert(lattice_a_idx, c_over_a_ratio_col)
cols.insert(lattice_a_idx + 1, b_over_a_ratio_col)


# Reindex the DataFrame with the new column order
summary_docs_df = summary_docs_df[cols]

summary_docs_df

In [ ]:
summary_docs_df = summary_docs_df.drop(columns=['symmetry_crystal_system', 'number_of_atoms', 'unit_cell_volume',
                                                'lattice_a', 'lattice_b', 'lattice_c'])

summary_docs_df

In [ ]:
print(surface_properties_docs)

In [ ]:
# Extract surface properties for all materials
surface_properties_lists = []
for doc in surface_properties_docs:
    for surface in doc.surfaces:
        surface_properties_lists.append({
            'material_id': doc.material_id,
            'pretty_formula': doc.pretty_formula,
            'work_function': surface.work_function,
            'miller_index': surface.miller_index,
            'surface_energy': surface.surface_energy,
            'fermi_energy': surface.efermi
        })

surface_properties_df = pd.DataFrame(surface_properties_lists)

surface_properties_df

In [ ]:
# Merge the two dataframes
merged_summary_surface_df_filtered = pd.merge(
    surface_properties_df,
    summary_docs_df,
    left_on=['material_id'],
    right_on=['material_id'],
    how='inner'  # Use 'inner' merge to keep only rows that match in both dataframes
)

merged_summary_surface_df_filtered

In [ ]:
print(sifat_atomik)

In [ ]:
atomic_properties_df = pd.DataFrame(sifat_atomik)

atomic_properties_df

In [ ]:
all_merged_df = pd.merge(
    merged_summary_surface_df_filtered,
    atomic_properties_df,
    left_on = ['pretty_formula'],
    right_on = ['formula_pretty'],
    how = 'left'
)

all_merged_df = all_merged_df.drop(columns = ['formula_pretty'])

all_merged_df['miller_index'] = all_merged_df['miller_index'].astype(str)
all_merged_df['structure_type'] = all_merged_df['structure_type'].astype(str)
all_merged_df['symmetry_symbol'] = all_merged_df['symmetry_symbol'].astype(str)

all_merged_df

In [ ]:
all_merged_df.info()

In [ ]:
print(f"Unique structure_type: {all_merged_df['structure_type'].nunique()}")
print(f"Unique miller_index: {all_merged_df['miller_index'].apply(tuple).nunique()}")
print(f"Unique symmetry_symbol: {all_merged_df['symmetry_symbol'].nunique()}")

In [21]:
all_merged_df.to_csv("metal_surface_properties_full_dataset.csv", index=False)

### Data Splitting

In [ ]:
y = all_merged_df['work_function']
X = all_merged_df.drop(columns=['work_function', 'material_id'])
groups = all_merged_df['pretty_formula']

X_features = X.drop(columns=['pretty_formula'])



outer_cv = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(outer_cv.split(X, y, groups=groups))

X_train_outer = X_features.iloc[train_idx]
y_train_outer = y.iloc[train_idx]
groups_train_outer = groups.iloc[train_idx]

X_test_outer = X_features.iloc[test_idx]
y_test_outer = y.iloc[test_idx]

print("--- Data Split Selesai (Versi Paling Aman) ---")
print(f"Jumlah data train (luar): {len(X_train_outer)}")
print(f"Jumlah data test (luar): {len(X_test_outer)}")

In [ ]:
print("--- Statistik y_train_outer ---")
print(y_train_outer.describe())

## Model's Training, Validation, Hyperparameter Tuning, Evaluation, and Interpretation

### Model's pipeline

In [ ]:
numeric_features_impute = ['Youngs_modulus', 'shear_modulus', 'bulk_modulus']
numeric_features_clean = [col for col in X_features.columns
                            if col.startswith(('surface', 'fermi', 'atomic_density',
                                                'c_over', 'b_over', 'lattice', 'atomic_number',
                                                'atomic_radius', '1st_ionization', 'electron_affinity',
                                                'electronegativity', 'valence_electrons'))]
numeric_features = numeric_features_impute + numeric_features_clean

categorical_features = ['miller_index', 'structure_type', 'symmetry_symbol']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', SVR(kernel='rbf'))
])

inner_cv = GroupKFold(n_splits=10)

scoring_metrics = {
    'r2': 'r2',
    'mae': 'neg_mean_absolute_error',
    'rmse': 'neg_root_mean_squared_error'
}

### Model's Training, Validation, and Hyperparameter Tuning

In [ ]:
coarse_param_grid = {
    'model__C': [0.01, 0.1, 1, 10, 100],
    'model__gamma': [0.01, 0.1, 'scale', 1, 10],
    'model__epsilon': [0.01, 0.05, 0.1, 0.5, 1]
}

print("--- Coarse Grid Didefinisikan ---")
print(coarse_param_grid)
print("Mulai Coarse GridSearchCV (Inner Loop)...")

grid_search_coarse = GridSearchCV(
    estimator=full_pipeline,
    param_grid=coarse_param_grid,
    cv=inner_cv,

    # --- PERUBAHAN DI SINI ---
    scoring=scoring_metrics,  # <-- Pakai dictionary metrik
    refit='r2',               # <-- Tetap pilih model terbaik berdasarkan R²
    # -----------------------------

    n_jobs=-1
)

grid_search_coarse.fit(X_train_outer, y_train_outer, groups=groups_train_outer)

print("Coarse Tuning selesai.")
print(f"Parameter kasar terbaik: {grid_search_coarse.best_params_}")
print(f"Skor R² CV kasar terbaik (di dalam train): {grid_search_coarse.best_score_:.4f}")

# --- BARU: Tampilkan Hasil Lengkap ---
print("\n--- Hasil Lengkap Coarse Grid Search ---")
results_df_coarse = pd.DataFrame(grid_search_coarse.cv_results_)
print(results_df_coarse[[
    'params',
    'mean_test_r2',
    'mean_test_mae',
    'mean_test_rmse'
]].sort_values(by='mean_test_r2', ascending=False).head())

In [ ]:
# --- 5. Definisikan dan Jalankan Fine Grid Search ---

# Buat grid halus (fine grid) berdasarkan hasil coarse search
# (Kamu harus sesuaikan ini berdasarkan output Cell 4)
fine_param_grid = {
    'model__C': [0.025, 0.05, 0.1, 0.2, 0.4],
    'model__gamma': [0.0025, 0.005, 0.01, 0.02, 0.04],
    'model__epsilon': [0.0125, 0.025, 0.05, 0.1, 0.2]
}

print("--- Fine Grid Didefinisikan ---")
print(fine_param_grid)

# --- Jalankan GridSearchCV (FINE) ---
print("\nMulai Fine GridSearchCV (Inner Loop)...")
grid_search_fine = GridSearchCV(
    estimator=full_pipeline,
    param_grid=fine_param_grid, # <-- Pakai grid halus
    cv=inner_cv,

    # --- PERUBAHAN DI SINI ---
    scoring=scoring_metrics,
    refit='r2',
    # -----------------------------

    n_jobs=-1
)

grid_search_fine.fit(X_train_outer, y_train_outer, groups=groups_train_outer)

print("Fine Tuning selesai.")
print(f"Parameter halus terbaik: {grid_search_fine.best_params_}")
print(f"Skor R² CV halus terbaik (di dalam train): {grid_search_fine.best_score_:.4f}")

# --- BARU: Tampilkan Hasil Lengkap ---
print("\n--- Hasil Lengkap Fine Grid Search ---")
results_df_fine = pd.DataFrame(grid_search_fine.cv_results_)
print(results_df_fine[[
    'params',
    'mean_test_r2',
    'mean_test_mae',
    'mean_test_rmse'
]].sort_values(by='mean_test_r2', ascending=False).head())

In [ ]:
# --- 6. Definisikan dan Jalankan 2nd Fine Grid Search ---

# Buat grid halus kedua (2nd fine grid) berdasarkan hasil fine search
fine_param_grid_2nd = {
    'model__C': [0.2, 0.3, 0.4, 0.5, 0.6],
    'model__gamma': [0.0005, 0.0015, 0.0025, 0.0035, 0.0045],
    'model__epsilon': [0.005, 0.015, 0.025, 0.035, 0.045]
}

print("--- 2nd Fine Grid Didefinisikan ---")
print(fine_param_grid_2nd)

# --- Jalankan GridSearchCV (FINE) ---
print("\nMulai 2nd Fine GridSearchCV (Inner Loop)...")
grid_search_fine_2nd = GridSearchCV(
    estimator=full_pipeline,
    param_grid=fine_param_grid_2nd, # <-- Pakai grid halus
    cv=inner_cv,

    # --- PERUBAHAN DI SINI ---
    scoring=scoring_metrics,
    refit='r2',
    # -----------------------------

    n_jobs=-1
)

grid_search_fine_2nd.fit(X_train_outer, y_train_outer, groups=groups_train_outer)

print("2nd Fine Tuning selesai.")
print(f"Parameter halus terbaik: {grid_search_fine_2nd.best_params_}")
print(f"Skor R² CV halus terbaik (di dalam train): {grid_search_fine_2nd.best_score_:.4f}")

# --- BARU: Tampilkan Hasil Lengkap ---
print("\n--- Hasil Lengkap 2nd Fine Grid Search ---")
results_df_fine_2nd = pd.DataFrame(grid_search_fine_2nd.cv_results_)
print(results_df_fine_2nd[[
    'params',
    'mean_test_r2',
    'mean_test_mae',
    'mean_test_rmse'
]].sort_values(by='mean_test_r2', ascending=False).head())

### Model's Evaluation

In [ ]:
# Ambil model terbaik dari grid halus
best_model = grid_search_fine_2nd.best_estimator_

# 1. Dapatkan prediksi di data test
y_pred = best_model.predict(X_test_outer)

# 2. Hitung semua metrik
final_r2 = r2(y_test_outer, y_pred)
final_mae = mae(y_test_outer, y_pred)
final_rmse = rmse(y_test_outer, y_pred)

# 3. Tampilkan hasilnya
print("\n--- HASIL FINAL (Evaluasi di X_test_outer) ---")
print(f"Parameter final terbaik: {grid_search_fine.best_params_}")
print("-" * 30)
print(f"Koefisien Determinasi (R²): {final_r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {final_mae:.4f}  (satuan eV)")
print(f"Root Mean Squared Error (RMSE): {final_rmse:.4f}  (satuan eV)")

### Model's Interpretation

#### SHAP

##### Initialization

In [ ]:
# Inisialisasi JS untuk plot SHAP di notebook
shap.initjs()

# --- PENTING! ---
# Ambil model terbaik dari 2nd_fine_tune (PEMENANG KITA)
best_model = grid_search_fine_2nd.best_estimator_

# Pisahkan preprocessor dan model SVR-nya
preprocessor = best_model.named_steps['preprocessor']
svr_model = best_model.named_steps['model']

print("SHAP, model pemenang, dan preprocessor siap.")

In [ ]:
print("Memproses data train dan test...")
# Proses data mentah pakai preprocessor
X_train_processed = preprocessor.transform(X_train_outer)
X_test_processed = preprocessor.transform(X_test_outer)

# Ambil nama fitur SETELAH di-OHE (ini krusial untuk plot)
# Nama fiturnya akan jadi seperti: 'num__atomic_radius', 'cat__miller_index_(0, 0, 1)', dll.
feature_names = preprocessor.get_feature_names_out()

print(f"Data diproses. Jumlah total fitur: {len(feature_names)}")

In [ ]:
print("Membuat 2 jenis ringkasan background data (K-Means)...")

background_data1 = shap.kmeans(X_train_processed, 100)
background_data2 = shap.kmeans(X_train_processed, 200)

print("2 jenis ringkasan background data siap.")

In [ ]:
print("Membuat 3 jenis KernelExplainer...")

explainer1 = shap.KernelExplainer(svr_model.predict, background_data1) # explainer dengan background_data K-Means 100 data
explainer2 = shap.KernelExplainer(svr_model.predict, background_data2) # explainer dengan background_data K-Means 200 data
explainer3 = shap.KernelExplainer(svr_model.predict, X_train_processed) # explainer dengan seluruh data train sebagai background

print("3 jenis explainer sudah siap. Mulai menghitung SHAP values dari masing-masing explainer untuk data test...")

shap_values1 = explainer1.shap_values(X_test_processed) # SHAP values menggunakan explainer1
shap_values2 = explainer2.shap_values(X_test_processed) # SHAP values menggunakan explainer2
shap_values3 = explainer3.shap_values(X_test_processed) # SHAP values menggunakan explainer3

print("Perhitungan SHAP values selesai.")

# --- PENTING: Buat DataFrame untuk plot ---
# Ubah data test yang sudah diproses jadi DataFrame
# Ini agar plot SHAP punya nama kolom yang benar
X_test_processed_df = pd.DataFrame(X_test_processed, columns=feature_names)

In [ ]:
# --- PLOT 1: Global Feature Importance (Bar Plot) ---
# Menunjukkan rata-rata dampak absolut. Fitur apa yang PALING PENTING?
print("--- Global Feature Importance (Bar) ---")
shap.summary_plot(shap_values, X_test_processed_df, plot_type="bar")

In [ ]:
# --- PLOT 2: Beeswarm Summary Plot ---
# Menunjukkan SETIAP data poin.
# - Sumbu X = Nilai SHAP (dampak ke prediksi)
# - Warna = Nilai fitur (Merah=Tinggi, Biru=Rendah)
print("\n--- SHAP Summary Plot (Beeswarm) ---")
shap.summary_plot(shap_values, X_test_processed_df)

In [ ]:
# --- PLOT 3: Waterfall Plot (VERSI BARU YANG SUDAH DIPERBAIKI) ---

# Kamu bisa ganti `idx_to_explain` ke angka lain (misal: 5, 10, 20)
# untuk melihat data poin lain di test set.
idx_to_explain = 0

print(f"--- Menampilkan Waterfall Plot untuk data poin ke-{idx_to_explain} ---")

# --- PERBAIKANNYA DI SINI ---
# 1. Buat 'Explanation' object secara manual untuk SATU data poin
exp_one_sample = shap.Explanation(
    values=shap_values[idx_to_explain],
    base_values=explainer.expected_value,
    data=X_test_processed_df.iloc[idx_to_explain].values, # Ambil nilainya
    feature_names=X_test_processed_df.columns.tolist() # Ambil nama fiturnya
)

# 2. Sekarang, panggil waterfall_plot HANYA dengan SATU object itu
shap.waterfall_plot(exp_one_sample)

In [ ]:
# --- PLOT 4: Interactive Force Plot (untuk SEMUA data test) ---
# Ini adalah plot yang "bisa digerakkan" (interactive)

print("--- Menampilkan Force Plot (Interaktif) ---")
# Catatan: Ini mungkin tidak bekerja di semua environment (misal Google Colab)
# tapi akan bekerja sempurna di Jupyter Notebook/Lab
shap.force_plot(
    explainer.expected_value, 
    shap_values, 
    X_test_processed_df
)